组归一化-Grop Normalization
===

# 1.批归一化的问题

批归一化(Batch Norm/BN)是深度学习中非常有效的一个技术，极大地推进了计算机视觉以及之外领域的前沿。BN通过计算一个(迷你)批量中的均值与方差来进行特征归一化。众多实践证明，它利于优化且使得深度网络易于收敛。批统计的随机不确定性也作为一个有利于泛化的正则化项。BN 已经成为了许多顶级计算机视觉算法的基础

尽管取得了很大的成果，BN也会因为归一不同批尺寸的独特行为而有缺点。特别是，BN需要用到足够大的批大小(例如，每个工作站采用32的批量大小)。一个小批量会导致估算批统计不准确，减小BN的批大小会极大地增加模型错误率。结果导致，如今许多模型都使用较大的批训练，它们非常耗费内存。反过来，训练模型时对BN效力的极度依赖性阻碍了人们用有限内存探索更高容量的模型。
![Images](Images/03/00/08_01_001.jpeg)

# 2.组归一化

In [ ]:
import torch
import torch.nn as nn